<a href="https://colab.research.google.com/github/ElenaS97/University-Workshops/blob/main/nlp_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import keras
import os
from pathlib import Path
from tensorflow import keras

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving rus.txt to rus.txt


Configuration

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
dirpath = Path(r"D:\OneDrive - hull.ac.uk\Trimester 2\Applied AI\w8-nlp_translation\rus.txt").parent.absolute()
data_path = os.path.join(dirpath, "rus.txt")

Prepare the data

In [ ]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype="float32",
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 10000
Number of unique input tokens: 72
Number of unique output tokens: 91
Max sequence length for inputs: 13
Max sequence length for outputs: 60


Build the model

In [ ]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

Train the model

In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s_model.keras")

Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 43s 323ms/step - accuracy: 0.7486 - loss: 1.3678 - val_accuracy: 0.7627 - val_loss: 0.9203
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 42s 334ms/step - accuracy: 0.7910 - loss: 0.8209 - val_accuracy: 0.7639 - val_loss: 0.9282
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 82s 333ms/step - accuracy: 0.8049 - loss: 0.7251 - val_accuracy: 0.7938 - val_loss: 0.7464
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 42s 333ms/step - accuracy: 0.8217 - loss: 0.6640 - val_accuracy: 0.8046 - val_loss: 0.6854
Epoch 5/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 40s 317ms/step - accuracy: 0.8322 - loss: 0.6058 - val_accuracy: 0.8145 - val_loss: 0.6640
Epoch 6/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 40s 317ms/step - accuracy: 0.8371 - loss: 0.5778 - val_accuracy: 0.8212 - val_loss: 0.6171
Epoch 7/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 41s 317ms/step - accuracy: 0.8450 - loss: 0.5456 - val_accuracy: 0.8338 - val_loss: 0.5908
Epoch 8/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 42s 334ms/step - accuracy: 0.8496 -

Run inference (sampling)  
1. encode input and retrieve initial decoder state  
2. run one step of decoder with this initial state and a "start of sequence" token as target. Output will be the next target token.  
3. Repeat with the current target token and current states

In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s_model.keras")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

You can now generate decoded sentences as such:

In [ ]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Прошилайай кома.

-
Input sentence: Go.
Decoded sentence: Прошилайай кома.

-
Input sentence: Go.
Decoded sentence: Прошилайай кома.

-
Input sentence: Hi.
Decoded sentence: Здрасте.

-
Input sentence: Hi.
Decoded sentence: Здрасте.

-
Input sentence: Hi.
Decoded sentence: Здрасте.

-
Input sentence: Hi.
Decoded sentence: Здрасте.

-
Input sentence: Hi.
Decoded sentence: Здрасте.

-
Input sentence: Hi.
Decoded sentence: Здрасте.

-
Input sentence: Run!
Decoded sentence: Беги!

-
Input sentence: Run!
Decoded sentence: Беги!

-
Input sentence: Run.
Decoded sentence: Бегите.

-
Input sentence: Run.
Decoded sentence: Бегите.

-
Input sentence: Run.
Decoded sentence: Бегите.

-
Input sentence: Run.
Decoded sentence: Бегите.

-
Input sentence: Who?
Decoded sentence: Кто такой его?

-
Input sentence: Wow!
Decoded sentence: За тол!

-
Input sentence: Wow!
Decoded sentence: За тол!

-
Input sentence: Wow!
Decoded sentence: За тол!

-
Input sentence: Wow!


GRU

In [ ]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.GRU(latent_dim, return_state=True)
encoder_outputs, state_h = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well.
decoder_gru = keras.layers.GRU(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, state_h = decoder_gru(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s3s_model.keras")

Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 42s 310ms/step - accuracy: 0.7498 - loss: 1.4655 - val_accuracy: 0.7723 - val_loss: 0.8943
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 38s 290ms/step - accuracy: 0.8029 - loss: 0.7842 - val_accuracy: 0.7916 - val_loss: 0.7469
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 38s 306ms/step - accuracy: 0.8218 - loss: 0.6588 - val_accuracy: 0.8061 - val_loss: 0.6712
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 39s 293ms/step - accuracy: 0.8335 - loss: 0.5917 - val_accuracy: 0.8210 - val_loss: 0.6243
Epoch 5/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 41s 293ms/step - accuracy: 0.8424 - loss: 0.5560 - val_accuracy: 0.8284 - val_loss: 0.5939
Epoch 6/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 41s 293ms/step - accuracy: 0.8474 - loss: 0.5329 - val_accuracy: 0.8324 - val_loss: 0.5785
Epoch 7/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 36s 292ms/step - accuracy: 0.8507 - loss: 0.5158 - val_accuracy: 0.8340 - val_loss: 0.5738
Epoch 8/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 37s 292ms/step - accuracy: 0.8520 -

In [ ]:
# Load trained model
model = keras.models.load_model("s3s_model.keras") # Load the GRU model

# === Encoder model (GRU returns only one output) ===
encoder_inputs = model.input[0]
state_h_enc = model.layers[2].output  # just one output: last hidden state
encoder_model = keras.Model(encoder_inputs, state_h_enc)

# === Decoder model (no state output, only sequences) ===
# Define the input for the decoder's sequence
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
# Define the input for the decoder's initial state
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="decoder_state_input_h")


# Get the GRU layer and the Dense layer from the trained model
decoder_gru = model.layers[3]
decoder_dense = model.layers[4]

# Pass the decoder inputs and initial state to the GRU layer
decoder_outputs, state_h_dec = decoder_gru(
    decoder_inputs, initial_state=decoder_state_input_h
)
# Pass the GRU outputs to the Dense layer
decoder_outputs = decoder_dense(decoder_outputs)

# Define the decoder model for inference
# It takes the decoder input sequence and the previous hidden state
# It outputs the predicted token probabilities and the new hidden state
decoder_model = keras.Model(
    [decoder_inputs, decoder_state_input_h], [decoder_outputs, state_h_dec]
)


def decode_sequence(input_seq):
    # Encode input as state
    state_value = encoder_model.predict(input_seq, verbose=0)
    if isinstance(state_value, list):
        state_value = state_value[0]  # take first tensor if list

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    decoded_sentence = ""
    stop_condition = False
    while not stop_condition:
        print("Shapes of inputs to decoder_model.predict():")
        print("target_seq shape:", target_seq.shape)
        print("state_value shape:", state_value.shape)

        output_tokens, h = decoder_model.predict(
            [target_seq, state_value], verbose=0
        )

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0
        state_value = h

    return decoded_sentence


# Test on a few samples
for seq_index in range(1): # Reduced to 1 sample for quicker debugging
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

Shapes of inputs to decoder_model.predict():
target_seq shape: (1, 1, 91)
state_value shape: (1, 256)
Shapes of inputs to decoder_model.predict():
target_seq shape: (1, 1, 91)
state_value shape: (1, 256)
Shapes of inputs to decoder_model.predict():
target_seq shape: (1, 1, 91)
state_value shape: (1, 256)
Shapes of inputs to decoder_model.predict():
target_seq shape: (1, 1, 91)
state_value shape: (1, 256)
Shapes of inputs to decoder_model.predict():
target_seq shape: (1, 1, 91)
state_value shape: (1, 256)
Shapes of inputs to decoder_model.predict():
target_seq shape: (1, 1, 91)
state_value shape: (1, 256)
Shapes of inputs to decoder_model.predict():
target_seq shape: (1, 1, 91)
state_value shape: (1, 256)
Shapes of inputs to decoder_model.predict():
target_seq shape: (1, 1, 91)
state_value shape: (1, 256)
Shapes of inputs to decoder_model.predict():
target_seq shape: (1, 1, 91)
state_value shape: (1, 256)
Shapes of inputs to decoder_model.predict():
target_seq shape: (1, 1, 91)
state_va